# Using MathLink to call mathematica
  ### The input expression is generated form the mathematica. Then it transformed to julia function or symEngine function. At the end it  will call the cuba program to integrate the functions numerically. The final output is the integral value
  Conclusion: julia function is much more faster then symEngine function. SymEngine function is as fast as Mathematica NIntegrate method. 

In [1]:
using SymEngine
using SyntaxTree
using Cuba, SpecialFunctions
using MathLink
function math2symEngine(symb::MathLink.WSymbol)
    SymEngine.symbols(symb.name)
end
function math2symEngine(num::Int)
    num
end
function math2symEngine(num::Number)
    num
end
function math2symEngine(expr::MathLink.WExpr)
    if expr.head.name=="Times"
        return *(map(math2symEngine,expr.args)...)
    elseif expr.head.name=="Plus"
        return +(map(math2symEngine,expr.args)...)
    elseif expr.head.name=="Power"
        return ^(map(math2symEngine,expr.args)...)
    elseif expr.head.name=="Rational"
        return  //(map(math2symEngine,expr.args)...)
    else
        return SymEngine.SymFunction(expr.head.name)(map(math2symEngine,expr.args)...)
    end
end
#Mathematica to julia expr
function math2Expr(symb::MathLink.WSymbol)
    Symbol(symb.name)
end
function math2Expr(num::Number)
    num
end
function math2Expr(expr::MathLink.WExpr)
    if expr.head.name=="Times"
        return Expr(:call, :*, map(math2Expr,expr.args)...)
    elseif expr.head.name=="Plus"
        return Expr(:call, :+,map(math2Expr,expr.args)...)
    elseif expr.head.name=="Power"
        return Expr(:call, :^, map(math2Expr,expr.args)...)
    elseif expr.head.name=="Rational"
        return  Expr(:call, ://, map(math2Expr,expr.args)...)
    else
        return Expr(:call, Symbol(expr.head.name), map(math2Expr,expr.args)...)
    end
end
function myconvert(::Type{Expr},ex::Basic)
    Expr(:call, :*, Symbol(SymEngine.toString(ex)), 1)
end
function evalSym(ex::SymEngine.Basic)
    fn = SymEngine.get_symengine_class(ex)     
    if fn == :FunctionSymbol
        as=get_args(ex)
        return Expr(:call, Symbol(get_name(ex)), [evalSym(a) for a in as]...)|>eval
    elseif fn == :Symbol
        return Symbol(SymEngine.toString(ex))|>eval
    elseif (fn in SymEngine.number_types) || (fn == :Constant)    
        return N(ex)|>eval
    elseif fn==:Mul
        as=get_args(ex)
        return *([evalSym(a) for a in as]...) 
    elseif fn==:Add
        as=get_args(ex)
        return +([evalSym(a) for a in as]...) 
    elseif fn==:Pow
        as=get_args(ex)
        return ^([evalSym(a) for a in as]...)
    elseif fn==:Rational
        as=get_args(ex)
        return //([evalSym(a) for a in as]...)
    end
end
macro genfun(expr,args)
    :($(Expr(:tuple,args.args...))->$expr)
end
genfun(expr,args) = :(@genfun $expr [$(args...)]) |> eval

genfun (generic function with 1 method)

In [57]:
#Copy the mathematica input form to W``` ......```
MLExpr=W```2^(-4 + DD) s23^(-4 + DD)
  Gamma[4 - DD](\[Alpha][2] + \[Alpha][3])^(4 - (3 DD)/2) (1 + \[Alpha][4])^(
 4 - (3 DD)/
  2) (-tt (\[Alpha][2] + \[Alpha][3]) + \[Alpha][2] \[Alpha][
     3] \[Alpha][4])^(-4 + DD)/.{DD->3.6,s23->1,tt->0.01}
/.{Gamma->gamma,\[Alpha][i_]:> ToExpression[StringJoin[{{ToString[\[Alpha]]},ToString[i]}]],Pi->pi}```|>weval

W"Times"(1.6810505838183376, W"Power"(W"Plus"(W"α2", W"α3"), -1.4000000000000004), W"Power"(W"Plus"(1, W"α4"), -1.4000000000000004), W"Power"(W"Plus"(W"Times"(-0.01, W"Plus"(W"α2", W"α3")), W"Times"(W"α2", W"α3", W"α4")), -0.3999999999999999))

In [58]:
symExpr=math2symEngine(MLExpr) #Transform from mathematica MathLink output to symEngine expression
juExpr=math2Expr(MLExpr) #Transform from mathematica MathLink output to Julia expression

:(1.6810505838183376 * (α2 + α3) ^ -1.4000000000000004 * (1 + α4) ^ -1.4000000000000004 * (-0.01 * (α2 + α3) + α2 * α3 * α4) ^ -0.3999999999999999)

In [59]:
symfun=lambdify(math2symEngine(tb),(:α2,:α3,:α4)) #Transform to the symEgnine function
jufun=genfun(juExpr,[:α2,:α3,:α4]) #Transform to the julia function 

#9 (generic function with 1 method)

In [60]:
symfun(0.2,0.2,0.2)|>print;print(",")
jufun(0.2,0.2,0.2)|>print

42.75816442008083,42.75816442008083

In [61]:
function intSec1t(x, f)#α1 sector decomposition
    tmp=tsym(Complex.(x)...)
    f[1],f[2]=reim(tmp)
end

intSec1t (generic function with 1 method)

In [62]:
function intSec1s(x, f)#α1 sector decomposition   
    tmp=jufun(Complex.(x)...)
    f[1],f[2]=reim(tmp)
end

intSec1s (generic function with 1 method)

In [63]:
@time int1numt = cuhre(intSec1t, 3, 2, atol=1e-10, rtol=1e-10);

  2.923385 seconds (24.15 M allocations: 953.531 MiB, 3.98% gc time)


In [66]:
@time int1nums = cuhre(intSec1s, 3, 2, atol=1e-10, rtol=1e-10);

  0.506695 seconds (12.00 M allocations: 488.343 MiB, 13.33% gc time)


In [55]:
int1numCt = int1numt[1][1]+int1numt[1][2]im 

17.79313098845209 - 26.214856183499197im

In [56]:
int1numCs = int1nums[1][1]+int1nums[1][2]im 

17.79313098845209 - 26.214856183499197im